In [236]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter

In [237]:
no_transactions = 2000 # will probably be random
no_buy_transactions = 800 #
no_sell_transactions  = 600 # also will probably be random
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [238]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [239]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([len(clients_cars) , len(our_cars)*50, len(clients_cars)/5])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [240]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [241]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

# klienci

In [242]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        pesel, birth_date = random_pesel_date_man(year_lims)
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        pesel, birth_date = random_pesel_date_woman(year_lims)
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]

In [243]:
clients_df

,id,imię,nazwisko,data_urodzenia,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Jan,Bogusławski,21/04/1973,M,73042144078,Wrocław,ul. Maxa Factora 21,False,48912535167,jan_bogusl2630@gmail.com
1,2,Filip,Legenza,09/07/2002,M,02270958638,Wrocław,ul. Nowa 21,False,48591713586,fil_legenz@gmail.com
2,3,Kinga,Karpińska,14/05/1998,F,98051412882,Ostrowiec Świętokrzyski,ul. Wilcza 85,False,48264700094,karpinska1@gmail.com
3,4,Ewelina,Rygiel,22/05/1984,F,84052210628,Wrocław,ul. Krasińskiego 42/20,False,48803637348,rygiel83045@wp.pl
4,5,Zofia,Mazur,16/11/1983,F,83111655787,Wrocław,ul. Jedności Robotniczej 21/37,False,48123110319,zof.mazu9615@outlook.com
...,...,...,...,...,...,...,...,...,...,...,...
1513,1514,Rafał,Lahouski,10/09/1988,M,88091044736,Łódź,ul. Jeżynowa 24/49,False,48583582573,raf-laho4961@gmail.com
1514,1515,Maria,Czogała,30/03/2008,F,08233007405,Gołdap,ul. Mennonicka 49/15,False,48792804199,m_czogal@vp.pl
1515,1516,Olha,Wolna,06/09/1993,F,93090660308,Wrocław,ul. Ogrodowa 70/28,False,48841621621,o.woln193@gmail.com
1516,1517,Ewa,Szymańska,30/06/1982,F,82063069808,Wrocław,ul. Morskich Fal 30/12,False,48656699932,szymanska2182@interia.pl


# auta